In [1]:
#!/bin/bash
!kaggle datasets download murtozalikhon/brain-tumor-multimodal-image-ct-and-mri

Dataset URL: https://www.kaggle.com/datasets/murtozalikhon/brain-tumor-multimodal-image-ct-and-mri
License(s): CC-BY-NC-SA-4.0
100% 359M/361M [00:14<00:00, 27.9MB/s]
100% 361M/361M [00:14<00:00, 26.8MB/s]


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
import cv2
import shutil

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
gpus = tf.config.experimental.list_physical_devices('gpu')
for gpu in gpus:
  tf.config.experimental.set_memory_growth('gpu', True)

In [5]:
!unzip '/content/brain-tumor-multimodal-image-ct-and-mri.zip'

Streaming output truncated to the last 5000 lines.
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (10).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (100).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1000).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1001).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1002).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1003).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1004).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1005).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1006).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1007).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1008).jpg  
  inflating: Dataset/Brain Tumo

In [6]:
new_dataset_healthy= '/content/NewDataset/HEALTHY'
new_dataset_tumor= '/content/NewDataset/TUMOR'

In [7]:
def new_folder_maker(path):
  if not os.path.exists(path):
    os.makedirs(path)

In [8]:
new_folder_maker(new_dataset_healthy)
new_folder_maker(new_dataset_tumor)

In [9]:
def images_mover(old_path, new_path):
  old_dataset_path=old_path
  new_dataset_path=new_path

  for img in os.listdir(old_dataset_path):
    old_data_path= os.path.join(old_dataset_path, img)
    new_data_path= os.path.join(new_dataset_path, img)

    shutil.copy(old_data_path, new_data_path)



In [10]:
#FOR CT SCAN MRI
images_mover(old_path='/content/Dataset/Brain Tumor CT scan Images/Healthy', new_path='/content/NewDataset/HEALTHY')

In [11]:
#FOR CT SCAN TUMOR
images_mover(old_path='/content/Dataset/Brain Tumor CT scan Images/Tumor', new_path='/content/NewDataset/TUMOR')

In [12]:
#FOR MRI SCAN HEALTHY
images_mover(old_path='/content/Dataset/Brain Tumor MRI images/Healthy', new_path='/content/NewDataset/HEALTHY')

In [13]:
#FOR MRI SCAN TUMOR
images_mover(old_path='/content/Dataset/Brain Tumor MRI images/Tumor', new_path='/content/NewDataset/TUMOR')

In [14]:
#checking if it was all the images that was transferred from the old filepath to the new filepath(HEALTHY)
dir, path, files= next(os.walk('/content/NewDataset/HEALTHY'))
len(files)

4300

In [15]:
#checking if it was all the images that was transferred from the old filepath to the new filepath(TUMOR)
dir, path, files= next(os.walk('/content/NewDataset/TUMOR'))
len(files)

5318

In [16]:
basedir='/content/NewDataset'
batch_size=32

In [21]:
datagen= ImageDataGenerator(rescale=1/255,
                            preprocessing_function=None,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            rotation_range=2,
                            validation_split=0.2
                            )

In [22]:
train_gen= datagen.flow_from_directory(
    basedir,
    batch_size= batch_size,
    class_mode='categorical',
    target_size=(128,128),
    subset='training',
)

Found 7695 images belonging to 2 classes.


In [54]:
for img_batch, label_batch in train_gen:
    print("Image batch shape:", img_batch.shape)
    print("Label batch shape:", label_batch.shape)
    break


Image batch shape: (32, 128, 128, 3)
Label batch shape: (32, 2)


In [23]:
val_gen= datagen.flow_from_directory(
    basedir,
    batch_size= batch_size,
    class_mode='categorical',
    target_size=(128,128),
    subset='validation',
)

Found 1923 images belonging to 2 classes.


In [24]:
from tensorflow.keras import layers, models

In [31]:
model= models.Sequential()
model.add(layers.Conv2D(64, (3,3), activation='relu', input_shape=(128,128,3)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(128, (3,3), activation='relu', input_shape=(128,128,3)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(2, activation='sigmoid'))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [32]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 126, 126, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 124, 124, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 58, 58, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 58, 58, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 29, 29, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 107648)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │         215,298 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 475,458 (1.81 MB)

 Trainable params: 475,458 (1.81 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']
              )

In [34]:
model.fit(train_gen, validation_data= val_gen, epochs=10)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


241/241 ━━━━━━━━━━━━━━━━━━━━ 85s 299ms/step - accuracy: 0.5487 - loss: 0.6896 - val_accuracy: 0.6859 - val_loss: 0.6119
Epoch 2/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 63s 258ms/step - accuracy: 0.7240 - loss: 0.5488 - val_accuracy: 0.8388 - val_loss: 0.3867
Epoch 3/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 82s 256ms/step - accuracy: 0.8140 - loss: 0.4020 - val_accuracy: 0.8508 - val_loss: 0.3682
Epoch 4/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 91s 298ms/step - accuracy: 0.8464 - loss: 0.3455 - val_accuracy: 0.8762 - val_loss: 0.3245
Epoch 5/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 74s 263ms/step - accuracy: 0.8713 - loss: 0.2935 - val_accuracy: 0.8534 - val_loss: 0.3341
Epoch 6/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 83s 269ms/step - accuracy: 0.8658 - loss: 0.2968 - val_accuracy: 0.8523 - val_loss: 0.3760
Epoch 7/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 80s 259ms/step - accuracy: 0.8901 - loss: 0.2811 - val_accuracy: 0.8934 - val_loss: 0.3168
Epoch 8/10
241/241 ━━━━━━━━━━━━━━━━━━━━ 74s 298ms/step - accuracy: 0.8899 - loss: 0.2643 - val

In [35]:
model.save('BrainTumor.h5')

In [73]:
input_filepath='/content/Dataset/Brain Tumor CT scan Images/Healthy/ct_healthy (1001).jpg'
img=cv2.imread(input_filepath)
img_resized= cv2.resize(img, (128,128))
img_rescaled= img_resized/255
img_rescaled= np.expand_dims(img_rescaled, axis=0)

In [74]:
np.argmax(model.predict(img_rescaled))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


0